In [162]:
import pandas as pd

In [163]:
data_medecin = pd.read_excel("/content/export_medecins.xlsx")

In [164]:
import re
import numpy as np
def search_adresse(text):
  text = str(text)
  text = text.replace(" " ,"")
  pattern_email = r'([\w\.-]+@[A-Za-z\-]+\.(com|ma|fr|net|org|edu|gov|io|co))'

  match = re.search(pattern_email , text)
  if match :
    return True
  return False

def extract_adresse(text):
  text = str(text)
  if search_adresse(text):
    pattern_email = r'([\w\.-]+@[A-Za-z\-]+\.(com|ma|fr|net|org|edu|gov|io|co))'
    match = re.search(pattern_email , text)
    return match.group(1)

  return np.nan


search_adresse('omarb773_oumaize@my-company.com8484')
extract_adresse('omar.b773_oumaize@my-company.com8484')


'omar.b773_oumaize@my-company.com'

In [165]:
print("Null counts in adresse")
print(data_medecin['adresse'].isna().value_counts())
print()
print("Null counts in adresse after extracting")
print(data_medecin['adresse'].apply(extract_adresse).isna().value_counts() )


Null counts in adresse
adresse
True     1254
False     230
Name: count, dtype: int64

Null counts in adresse after extracting
adresse
True     1455
False      29
Name: count, dtype: int64


looking for valid email

In [166]:
found_adresse_columns = []
search_adresses_colums = data_medecin.columns.tolist()


data_medecin['id'] = data_medecin['id'].astype(str)



In [167]:
data_medecin['adresse'][data_medecin['adresse'].apply(search_adresse)]

,adresse
17,essakhifadil@gmail.com
36,boualamabdelaziz@gmail.com
46,celgani@hotmail.com
65,sabah.fadouach@gmail.com
80,celgani@hotmail.com
90,mustaphaelgani@gmail.com
142,zouhair_laaouina2016@outlook.fr
147,nguermai@hotmail.com
183,n.alouta@yahoo.fr
237,ADNANELAJI@gmail.com


In [168]:
columns= data_medecin.drop(columns=['adresse']).columns.tolist()
list_search = []

for col in columns :
  result = data_medecin[col][data_medecin[col].apply(search_adresse)]
  shape = result.shape[0]

  if shape > 1 :
    column = f"adresse_from{col}"
    list_search.append(column)
    data_medecin[column] = result

no adresse found

In [169]:
list_search

[]

telephone

In [170]:
def search_fax(text):
  if isinstance(text , str):
    text = text.replace(' ' , '')
    patterns_phone =r'\D'
    filtered = re.sub(patterns_phone,'' , text)
    extract_fax = re.search(r'(\d{9})' , filtered)
    if extract_fax :
      return True
  return False


def extract_fax(text):
  if isinstance(text , str):
    text = text.replace(' ' , '')
    patterns_phone =r'\D'
    filtered = re.sub(patterns_phone,'' , text)
    pattern_fax1 = re.search(r'(?:212)*[+*.:/,-]*(0\d{9})$' , filtered)
    pattern_fax2 = re.search(r'(?:212)*[+*.:/,-]*(\d{9})$' , filtered)

    if pattern_fax1 :
      return pattern_fax1.group(1)

    elif pattern_fax2 :
      return '0' + pattern_fax2.group(1)

  return np.nan
print(extract_fax('CIN : N 159477 / 670249049'))
print(extract_fax('CIN : N 159477 / 212+0670249049'))


0670249049
0670249049


In [171]:
print("phone in adresse")
print(data_medecin['adresse'].apply(search_fax).value_counts())
data_medecin['adresse'][data_medecin['adresse'].apply(search_fax)]

phone in adresse
adresse
False    1476
True        8
Name: count, dtype: int64


,adresse
20,OUNAGHA 0648700910
157,SAADA 0524463381
199,0662115855
262,WATS 0649360707
376,0676023869 KARIMA
753,0662118177
857,0522970118 ARRIVAGE
1073,0660800504


In [172]:
columns= data_medecin.columns.tolist()
list_search = []

for col in columns :
  result = data_medecin[col][data_medecin[col].apply(search_fax)]
  shape = result.shape[0]

  if shape > 1 :
    column = f"telephone_from_{col}"
    list_search.append(column)
    data_medecin[column] = result

In [173]:
data_medecin


for col in list_search :
  mask = data_medecin['telephone1'].isna() & data_medecin[col].notna()
  data_medecin.loc[mask , 'telephone1'] = data_medecin.loc[mask , col]


In [174]:
mask = data_medecin['telephone1'].dropna().apply(extract_fax).index

data_medecin.loc[mask , 'telephone1'] = data_medecin['telephone1'].loc[mask]

In [175]:
data_medecin['telephone1'].dropna()

,telephone1
4,0524476910
5,0524472310
6,0524784806
7,0524784077
8,0524473872
...,...
1407,0649258888
1419,0524472535
1447,0524478080
1457,0524021286


groupage data

Table that stores information related to the blood types of patients.

In [176]:
data_groupage = pd.read_excel('/content/export_groupages.xlsx')

In [177]:
data_groupage.columns

Index(['id', 'ref', 'datevisite', 'nom', 'adresse', 'datenaiss',
       'Aff_datenaiss', 'determine1', 'determine2', 'Groupe', 'Rhesus',
       'Phenotype', 'Vignette', 'RefAutomate', 'groupe2', 'Rhesus2',
       'Phenotype2', 'sexe', 'Aff_Groupe', 'Vignette2', 'Rq1', 'Rq2', 'Rq3',
       'Rq4'],
      dtype='object')

Data we need to integrate

In [178]:
data_groupage[['id' , 'Groupe' , 'determine1' , 'determine2' ,'id' , 'Rhesus' , 'Vignette' ]]

,id,Groupe,determine1,determine2,id,Rhesus,Vignette
0,1,A,31/10/2014,NaN,1,POSITIF,A RH POSITIF
1,2,O,01/11/2014,NaN,2,POSITIF,O RH POSITIF
2,3,A,03/11/2014,NaN,3,POSITIF,A RH POSITIF
3,4,O,03/11/2014,NaN,4,POSITIF,O RH POSITIF
4,5,O,03/11/2014,NaN,5,POSITIF,O RH POSITIF
...,...,...,...,...,...,...,...
13309,13316,A,20/11/2024,NaN,13316,POSITIF,A+
13310,13317,O,20/11/2024,NaN,13317,POSITIF,O+
13311,13318,O,20/11/2024,NaN,13318,POSITIF,O+
13312,13319,A,13/07/2019,21/11/2024,13319,POSITIF,A+


Transforming Vignette

In [179]:
import re

def search_vignette(text):
    text = str(text).upper()
    # Don't remove spaces here, keep them for proper matching
    pattern = r'\bP\s*O\s*S\s*I\s*T\s*I\s*F\b|\bN\s*E\s*G\s*A\s*T\s*I\s*F\b|\bR\s*H\b'

    if re.search(pattern, text, flags=re.IGNORECASE):
        return True
    else:
        return False

def transform_vignette(text):
    if search_vignette(text):
        text = str(text).upper()
        # Don't remove spaces before substitution
        def repl(m):
            if m.group(1):  # POSITIF matched
                return '+'
            elif m.group(2):  # NEGATIF matched
                return '-'
            elif m.group(3):  # RH matched
                return ''
            else:
                return m.group(0)

        pattern = r'(\bP\s*O\s*S\s*I\s*T\s*I\s*F\b)|(\bN\s*E\s*G\s*A\s*T\s*I\s*F\b)|(\bR\s*H\b)'
        text = re.sub(pattern, repl, text, flags=re.IGNORECASE)

        # Now remove spaces after replacement
        text = text.replace(' ', '')
        return text

    else:
        return text  # or return original text if no match

print(transform_vignette('O RH POSITIF'))  # Expected output: O+
print(transform_vignette('A RH NEGATIF'))  # Expected output: A-


O+
A-


In [180]:
transformed_Vignette = data_groupage['Vignette'][data_groupage['Vignette'].apply(search_vignette) ].apply(transform_vignette)

data_groupage['Vignette'].loc[transformed_Vignette.index] = transformed_Vignette

<ipython-input-180-77cbe34ea656>:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data_groupage['Vignette'].loc[transformed_Vignette.index] = transformed_Vignette
<ipython-input-180-77cbe34ea656>:3: SettingWithCopyWarning: 
A value is trying

In [181]:
data_groupage['Vignette'].value_counts()

,count
Vignette,
O+,6094
A+,3932
B+,1738
O-,544
AB+,430
A-,363
B-,164
AB-,49


In [182]:
data_groupage[['determine1' , 'determine2' ]]

,determine1,determine2
0,31/10/2014,NaN
1,01/11/2014,NaN
2,03/11/2014,NaN
3,03/11/2014,NaN
4,03/11/2014,NaN
...,...,...
13309,20/11/2024,NaN
13310,20/11/2024,NaN
13311,20/11/2024,NaN
13312,13/07/2019,21/11/2024


In [183]:
data_groupage['Groupe'].value_counts()

,count
Groupe,
O,6638
A,4295
B,1902
AB,479


In [184]:
data_groupage['determine1'].str.extract(r'(\d{1,2}/\d{1,2}/\d{4})')

,0
0,31/10/2014
1,01/11/2014
2,03/11/2014
3,03/11/2014
4,03/11/2014
...,...
13309,20/11/2024
13310,20/11/2024
13311,20/11/2024
13312,13/07/2019


In [185]:
wow = '2014 hf \ 20 chhc \ 30'

def extract_date(text) :
  text = str(text).replace(' ' , '')
  text = re.sub('\D' , '' , text)
  match = re.search(r'(\d{2}/\d{1,2}/\d{4})' , text)
  if match:
    return match.group(1)

  else :
    return  text
extract_date(wow)

'20142030'

In [186]:
import re

def search_date(text):
    text = str(text) # Keep original spaces for initial date search
    match = re.search(r'(\d{2}).*(\d{2}).*(\d{4})', text)
    if match:
      return True

    return False


def extract_date(text):
    if search_date(text) :
      match = re.search(r'(\d{2}).*(\d{2}).*(\d{4})', text)
      return match.group(3) +'-' + match.group(2) + '-' + match.group(1)
    return text

extract_date('02DHDHHD/ 05 /GDGDG/HDH2014')

'2014-05-02'

In [187]:
data_groupage['determine1'] = data_groupage['determine1'].apply(extract_date)
data_groupage['determine1']

,determine1
0,2014-10-31
1,2014-11-01
2,2014-11-03
3,2014-11-03
4,2014-11-03
...,...
13309,2024-11-20
13310,2024-11-20
13311,2024-11-20
13312,2019-07-13


In [188]:
data_groupage['determine2'] = data_groupage['determine2'][ data_groupage['determine2'].str.contains(r'.*(\d{1,2}).*(\d{1,2}).*(\d{4})' , na=False) ].apply(extract_date)

<ipython-input-188-cd51398f45b6>:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data_groupage['determine2'] = data_groupage['determine2'][ data_groupage['determine2'].str.contains(r'.*(\d{1,2}).*(\d{1,2}).*(\d{4})' , na=False) ].apply(extract_date)


In [189]:
data_groupage['determine2'].dropna()

,determine2
391,2015-02-10
864,2015-05-14
1533,2015-09-02
2639,2021-05-19
4717,2021-05-17
...,...
13077,2024-08-05
13099,2024-08-24
13108,2024-08-29
13305,2024-11-19


In [190]:
data_groupage[['determine1' , 'determine2']].dropna()

,determine1,determine2
391,2015-02-10,2015-02-10
864,2015-05-09,2015-05-14
1533,2015-09-02,2015-09-02
2639,2016-02-23,2021-05-19
4717,2017-03-09,2021-05-17
...,...,...
13077,2021-05-19,2024-08-05
13099,2018-01-10,2024-08-24
13108,2018-08-16,2024-08-29
13305,2024-02-25,2024-11-19


join data

Nom du médecin prescripteur ou informations sur la prescription.

In [191]:
all_data_patient = pd.read_excel('/content/all_data_patient_final_clean.xlsx'  , dtype={'telephone': str})

In [192]:
all_data_patient.columns

Index(['Unnamed: 0', 'Chun', 'adresse', 'region', 'cin', 'dateheure', 'dnaiss',
       'email', 'nomprenom', 'ba', 'telephone', 'sit', 'observ', 'comment1',
       'comment2', 'ref', 'age', 'telephone_extracted_from_adresse',
       'cin_from_region', 'cin_from_adress', 'nom', 'prenom',
       'predicted_gender', 'year_naiss', 'total_comment'],
      dtype='object')

In [193]:
filter_all_data_patient = all_data_patient[['Chun','nom', 'prenom' , 'adresse', 'region', 'cin', 'dateheure', 'dnaiss',
       'email', 'ba', 'telephone', 'sit', 'ref', 'age' , 'total_comment']]

In [194]:
data_patient = pd.read_excel('/content/export_patients.xlsx')

In [195]:
filter_all_data_patient['medecin_nom'] = data_patient['prescri']

<ipython-input-195-bb7ad9dcef46>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_all_data_patient['medecin_nom'] = data_patient['prescri']


In [196]:
filter_all_data_patient.head(5)

,Chun,nom,prenom,adresse,region,cin,dateheure,dnaiss,email,ba,telephone,sit,ref,age,total_comment,medecin_nom
0,1,TRIFHAT,RKIA,NaN,NaN,NaN,2014-10-31 07:37:46,1958-01-01,0,F,0661425650,MME,14J1,56 ans,"TRAITEMENT ?: SINTROM,",DERHAM
1,2,CHBIKI,SALIMA,NaN,NaN,J485388,2014-10-31 07:49:36,1993-01-04,0,F,0607098707,MME,14J2,21 ans,"Grossesse ?: OUI,",ALAHYANE LATIFA
2,3,NAIMI,FATIMA,NaN,NaN,N105701,2014-10-31 07:58:42,1960-01-01,0,F,0666014056,MME,14J3,54 ans,,NaN
3,4,GHASSOUL,ZOHRA,NaN,NaN,NaN,2014-10-31 08:03:07,1961-12-12,0,F,0666594273,MME,14J4,52 ans,,KHABOUZI SAMIRA
4,5,EL KHARRIGO,TAOUFIQ,NaN,NaN,NaN,2014-10-31 08:06:34,1962-04-30,0,F,0666155610,MME,14J5,54 ans,,FADOUACH


In [197]:
data_medecin = data_medecin[['id','nomprenom','adresse' ,'region' ,'specialite', 'telephone1'	, 'telephone2'	, 'fax' ,	'gsm']]
data_medecin.head(5)

,id,nomprenom,adresse,region,specialite,telephone1,telephone2,fax,gsm
0,38,SLIMANI RACHID,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,35,ALAOUI MOHAMMED,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36,BERRADA ALI,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,37,BOUZIANE HASSAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40,EL HADDAD SAID,NaN,NaN,NaN,0524476910,NaN,0524476910,0661191998


joining with medecin nom

In [198]:
merged_data = filter_all_data_patient.merge(data_medecin , how='left' ,left_on='medecin_nom' , right_on='nomprenom' ,suffixes=('', '_doctor'))

In [199]:
merged_data.head(5)

,Chun,nom,prenom,adresse,region,cin,dateheure,dnaiss,email,ba,...,medecin_nom,id,nomprenom,adresse_doctor,region_doctor,specialite,telephone1,telephone2,fax,gsm
0,1,TRIFHAT,RKIA,NaN,NaN,NaN,2014-10-31 07:37:46,1958-01-01,0,F,...,DERHAM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,CHBIKI,SALIMA,NaN,NaN,J485388,2014-10-31 07:49:36,1993-01-04,0,F,...,ALAHYANE LATIFA,49,ALAHYANE LATIFA,NaN,NaN,NaN,0524785860,NaN,NaN,0661833670
2,3,NAIMI,FATIMA,NaN,NaN,N105701,2014-10-31 07:58:42,1960-01-01,0,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,GHASSOUL,ZOHRA,NaN,NaN,NaN,2014-10-31 08:03:07,1961-12-12,0,F,...,KHABOUZI SAMIRA,54,KHABOUZI SAMIRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,EL KHARRIGO,TAOUFIQ,NaN,NaN,NaN,2014-10-31 08:06:34,1962-04-30,0,F,...,FADOUACH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [200]:
columns = filter_all_data_patient.columns.tolist()
columns.extend(['id'])
merged_data = merged_data[columns]

merged_data

,Chun,nom,prenom,adresse,region,cin,dateheure,dnaiss,email,ba,telephone,sit,ref,age,total_comment,medecin_nom,id
0,1,TRIFHAT,RKIA,NaN,NaN,NaN,2014-10-31 07:37:46,1958-01-01,0,F,0661425650,MME,14J1,56 ans,"TRAITEMENT ?: SINTROM,",DERHAM,NaN
1,2,CHBIKI,SALIMA,NaN,NaN,J485388,2014-10-31 07:49:36,1993-01-04,0,F,0607098707,MME,14J2,21 ans,"Grossesse ?: OUI,",ALAHYANE LATIFA,49
2,3,NAIMI,FATIMA,NaN,NaN,N105701,2014-10-31 07:58:42,1960-01-01,0,F,0666014056,MME,14J3,54 ans,,NaN,NaN
3,4,GHASSOUL,ZOHRA,NaN,NaN,NaN,2014-10-31 08:03:07,1961-12-12,0,F,0666594273,MME,14J4,52 ans,,KHABOUZI SAMIRA,54
4,5,EL KHARRIGO,TAOUFIQ,NaN,NaN,NaN,2014-10-31 08:06:34,1962-04-30,0,F,0666155610,MME,14J5,54 ans,,FADOUACH,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257375,253982,KOUISS,HIND,NaN,NaN,N358582,2024-11-21 12:06:04,1992-12-25,0,F,0699334313,MLLE,15F2187,31 ans,,EL HADDAD SAID,40
257376,253983,ETTAKNI,ANDELLATIF,NaN,N26867,N26867,2024-11-21 12:10:09,1982-01-01,0,M,0610904616,MR,24K417,42 ans,"Abstinence: ,",KADIRI MOHAMED,352
257377,253984,CHAPIN,MARCEL,NaN,NaN,NaN,2024-11-21 12:15:53,1947-01-28,0,M,0648816169,MR,15B1355,77 ans,,BENALIL ASMA,69
257378,253985,KHIALI,FIRSAOUSS,KHIALI MOHAMED,NaN,NaN,2024-11-21 13:15:54,1911-02-23,0,F,0610088262,ENF,24K419,13 ans,,AYOUB AREGRAG,NaN


In [201]:
data_groupage = data_groupage[['Groupe' , 'determine1' , 'determine2' ,'id' , 'Rhesus' , 'Vignette' , 'ref' ]]

In [202]:
#merged_data = filter_all_data_patient.merge(data_medecin , how='left' ,left_on='medecin_nom' , right_on='nomprenom' ,suffixes=('', '_doctor'))
merged_columns= merged_data.columns.tolist()
merged_columns.extend(['Groupe' , 'determine1' , 'determine2' ,'id' , 'Rhesus' , 'Vignette' ])
merged_data =  merged_data.merge(data_groupage , how='left' , left_on='ref' , right_on='ref' , suffixes=('' , '_groupage'))



In [203]:
data_groupage

,Groupe,determine1,determine2,id,Rhesus,Vignette,ref
0,A,2014-10-31,NaN,1,POSITIF,A+,14J48
1,O,2014-11-01,NaN,2,POSITIF,O+,14K9
2,A,2014-11-03,NaN,3,POSITIF,A+,14K56
3,O,2014-11-03,NaN,4,POSITIF,O+,14K48
4,O,2014-11-03,NaN,5,POSITIF,O+,14K74
...,...,...,...,...,...,...,...
13309,A,2024-11-20,NaN,13316,POSITIF,A+,24K387
13310,O,2024-11-20,NaN,13317,POSITIF,O+,24K391
13311,O,2024-11-20,NaN,13318,POSITIF,O+,24C417
13312,A,2019-07-13,2024-11-21,13319,POSITIF,A+,19F335


In [204]:
grouped  =  data_groupage.sort_values(by = 'ref').groupby(['ref']).size().reset_index(name='size')
grouped[grouped['size']>1]

,ref,size
1,14K1008,2
26,14K189,2
48,14K435,2
67,14K650,2
73,14K708,2
...,...,...
11909,21K89,2
12104,22J172,2
12244,23H436,2
12296,23J340,2


In [205]:
data_groupage[data_groupage['ref']=='14K1008']

,Groupe,determine1,determine2,id,Rhesus,Vignette,ref
5346,AB,2017-07-03,NaN,5352,POSITIF,AB+,14K1008
10399,AB,2017-07-03,2020-06-30,10405,POSITIF,AB+,14K1008


In [206]:
if filter_all_data_patient[filter_all_data_patient['ref']== '14K1008'].shape[0] * data_groupage[  data_groupage['ref'] == '14K1008' ].shape[0] == merged_data[merged_data['ref'] == '14K1008'].shape[0] :
  print("True")

else :
  print("there's a probleme ")

True


In [207]:
data_patient.columns

Index(['ref', 'datevisite', 'dateheure', 'Heure', 'ba', 'sit', 'nomprenom',
       'cin', 'dnaiss', 'age', 'agejours', 'telephone', 'adresse', 'region',
       'observ', 'prescri', 'modalite', 'clinique', 'chambre', 'retrait1',
       'retrait1conv', 'retrait2', 'retrait2conv', 'heureretrait',
       'heureretrait2', 'recupere', 'daterecupere', 'convdaterecupere',
       'montantsanspreleve', 'typeprelevement', 'heure_preleve', 'nom_preleve',
       'prelevement', 'montantfinal', 'remise', 'montantfinalremise',
       'convdate', 'reelsanspreleve', 'prixreel', 'Chun', 'cotb',
       'RefAutomate', 'mutuelle', 'sav', 'mtsel', 'comment1', 'comment2',
       'Ch1', 'Num1', 'Imp_Retrait', 'Imp_Facture', 'valide', 'urgent',
       'ordreimp', 'email'],
      dtype='object')

In [208]:
pd.DataFrame(merged_data.columns , columns = ['column'] )

,column
0,Chun
1,nom
2,prenom
3,adresse
4,region
5,cin
6,dateheure
7,dnaiss
8,email
9,ba


In [209]:
isil = '''id
adresse1
adresse2
cin
code_postal
date_creation
date_modification
date_naissance
email
fax
fixe
nom
prenom
sexe
tel
username
ville
champsde_transmission
civilite
commentaire
ipp
mode_transmission
nom_jeune_fille
solde
canal_transmission_id
medecin_id
path_cin
gs
date_determ1
date_determ2
num_determ
ref_determ1
ref_determ2
rhesus
autre_identite
passeport
permis_conduire
titre_sejour
groupe_sanguin
responsable_legal_id'''
mogador = '''-
adresse
region
cin
-
dateheure
-
dnaiss
email
-
-
nomprenom
nomprenom
ba
telephone
-
-
-
sit
observ, comment1, comment2
ref
-
-
-
-
id
-
Groupe
determine1
determine2
id
-
-
Rhesus
-
-
-
-
Vignette
-'''

explication = '''Identifiant unique du patient.
Adresse principale du patient.
Adresse secondaire (si disponible).
Numéro de carte d'identité nationale.
Code postal de la ville du patient.
Date et heure de création de l'enregistrement.
Date et heure de la dernière modification.
Date de naissance du patient.
Adresse e-mail du patient.
Numéro de fax du patient.
Numéro de téléphone fixe.
Nom de famille du patient.
Prénom du patient.
Sexe du patient (M pour masculin, F pour féminin).
Numéro de téléphone mobile.
Nom d'utilisateur associé au patient.
Ville de résidence.
Champs utilisés pour la transmission des données.
Civilité du patient (MR, MME, etc.).
Commentaires supplémentaires.
Identifiant patient permanent.
Mode de transmission des documents médicaux.
Nom de jeune fille (pour les patientes mariées).
Solde du compte du patient (si applicable).
Identifiant du canal de transmission.
Identifiant du médecin référent.
Chemin du fichier contenant la copie de la CIN.
Groupe sanguin du patient.
Date de la première détermination du groupe sanguin.
Date de la deuxième détermination du groupe sanguin.
Numéro de détermination.
Référence de la première détermination.
Référence de la deuxième détermination.
Facteur rhésus du patient (+ ou -).
Autres identités utilisées par le patient.
Numéro de passeport.
Numéro du permis de conduire.
Numéro du titre de séjour (si étranger).
Groupe sanguin du patient (peut être redondant avec gs).
Identifiant du responsable légal (si applicable).'''
explication = explication.split('\n')

mogador = mogador.split('\n')
isil = isil.split('\n')
len(isil) , len(mogador)


dict_mapping = {'isil' : isil , 'mogador' : mogador, 'explication' : explication }

df = pd.DataFrame(dict_mapping)
df

,isil,mogador,explication
0,id,-,Identifiant unique du patient.
1,adresse1,adresse,Adresse principale du patient.
2,adresse2,region,Adresse secondaire (si disponible).
3,cin,cin,Numéro de carte d'identité nationale.
4,code_postal,-,Code postal de la ville du patient.
5,date_creation,dateheure,Date et heure de création de l'enregistrement.
6,date_modification,-,Date et heure de la dernière modification.
7,date_naissance,dnaiss,Date de naissance du patient.
8,email,email,Adresse e-mail du patient.
9,fax,-,Numéro de fax du patient.


In [210]:
raw_groupage = pd.read_excel('/content/export_groupages.xlsx')

In [211]:
pd.DataFrame(raw_groupage.columns , columns = ['columns'])

,columns
0,id
1,ref
2,datevisite
3,nom
4,adresse
5,datenaiss
6,Aff_datenaiss
7,determine1
8,determine2
9,Groupe


In [212]:
merged_data.columns

Index(['Chun', 'nom', 'prenom', 'adresse', 'region', 'cin', 'dateheure',
       'dnaiss', 'email', 'ba', 'telephone', 'sit', 'ref', 'age',
       'total_comment', 'medecin_nom', 'id', 'Groupe', 'determine1',
       'determine2', 'id_groupage', 'Rhesus', 'Vignette'],
      dtype='object')

In [213]:
merged_data['determine1'] =  pd.to_datetime(merged_data['determine1'] )
merged_data['determine2'] =  pd.to_datetime(merged_data['determine2'] )
merged_data['dateheure'] =  pd.to_datetime(merged_data['dateheure'] )
merged_data['dnaiss'] =  pd.to_datetime(merged_data['dnaiss'] )
merged_data['id'] = merged_data['id'].astype('Int64')



merged_data.dtypes

,0
Chun,int64
nom,object
prenom,object
adresse,object
region,object
cin,object
dateheure,datetime64[ns]
dnaiss,datetime64[ns]
email,int64
ba,object


In [214]:
merged_data['id']

,id
0,<NA>
1,49
2,<NA>
3,54
4,<NA>
...,...
259901,40
259902,352
259903,69
259904,<NA>


In [215]:
unique_data = merged_data.sort_values(by = ['ref' , 'dateheure'] , ascending = [True , False]).drop_duplicates(subset = ['ref'] , keep='first')

In [216]:
unique_data['id'] = unique_data['id'].apply(lambda x  : np.nan if pd.isna(x) else x )

In [217]:
unique_data.isna().sum()

,0
Chun,0
nom,35
prenom,35
adresse,85119
region,79866
cin,61134
dateheure,0
dnaiss,2
email,0
ba,0


In [218]:
unique_data[unique_data['sit'].isna()][['ref' , 'determine1']]

,ref,determine1
4422,15A1596,NaT
3581,15A888,NaT
8353,15C239,NaT
12672,15D1097,NaT
13402,15D1601,NaT
...,...,...
255467,24I130,NaT
255831,24I277,NaT
257022,24J104,NaT
256770,24J19,NaT


some of paients there are not in data groupage so they got null in determine1 and the others enregistrements

In [219]:
data_groupage[['ref' , 'determine1']][data_groupage['ref'] =='15A1596']

,ref,determine1


In [220]:
grouped = merged_data['ref'].value_counts().reset_index()
grouped[grouped['count'] == 5 ].sort_values(by=['count'] , ascending =True)

,ref,count
11739,24B408,5
11738,16K714,5
11737,15D74,5
11736,15D70,5
11735,14J21,5
...,...,...
11742,15D106,5
11743,15D95,5
11744,16K736,5
11745,15L237,5


In [221]:
merged_data[['ref' , 'id']][merged_data['ref'].isin(['24B408' , '15D106'])]

,ref,id
11293,15D106,<NA>
19957,15D106,<NA>
78861,15D106,<NA>
88054,15D106,<NA>
96453,15D106,<NA>
245190,24B408,86
252829,24B408,<NA>
258150,24B408,<NA>
259099,24B408,<NA>
259118,24B408,<NA>


In [222]:
unique_data.to_csv('merged_data.csv' , index=False)

In [224]:
data_patient.columns

Index(['ref', 'datevisite', 'dateheure', 'Heure', 'ba', 'sit', 'nomprenom',
       'cin', 'dnaiss', 'age', 'agejours', 'telephone', 'adresse', 'region',
       'observ', 'prescri', 'modalite', 'clinique', 'chambre', 'retrait1',
       'retrait1conv', 'retrait2', 'retrait2conv', 'heureretrait',
       'heureretrait2', 'recupere', 'daterecupere', 'convdaterecupere',
       'montantsanspreleve', 'typeprelevement', 'heure_preleve', 'nom_preleve',
       'prelevement', 'montantfinal', 'remise', 'montantfinalremise',
       'convdate', 'reelsanspreleve', 'prixreel', 'Chun', 'cotb',
       'RefAutomate', 'mutuelle', 'sav', 'mtsel', 'comment1', 'comment2',
       'Ch1', 'Num1', 'Imp_Retrait', 'Imp_Facture', 'valide', 'urgent',
       'ordreimp', 'email'],
      dtype='object')

In [228]:
data_patient['region'].apply(search_adresse).value_counts()


,count
region,
False,253565
